In [46]:
import torch
from transformers import pipeline, AutoModelForCausalLM
import torch
from peft import PeftModel
import textwrap
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
from transformers.generation.utils import GreedySearchDecoderOnlyOutput

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


MODEL = 'beomi/KoAlpaca'

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device="cuda", non_blocking=True)
model.eval()

tokenizer = LlamaTokenizer.from_pretrained("beomi/KoAlpaca")

PROMPT_TEMPLATE = f"""
아래의 요청사항에 따라 입력된 문장을 재구성 해주세요
### 입력된 문장:[instruction]
### 요청사항:
1. 다양성을 극대화하기 위해 입련된 문장과 다른 동사를 사용해 문장을 구성해주세요.
2. 답변은 한국어로 작성해야 합니다.
3. 답변을 뉴스 기사 제목처럼 제작 해주어야 합니다. 문장 수는 2개 이내로 구성해주세요

### Response:

"""
def create_prompt(instruction: str) -> str:
    return PROMPT_TEMPLATE.replace("[instruction]", instruction)

def generate_response(prompt: str, model: PeftModel) -> GreedySearchDecoderOnlyOutput:
    encoding = tokenizer(prompt, return_tensors="pt")
    input_ids = encoding["input_ids"].to(DEVICE)
 
    generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.75,
        repetition_penalty=1.1,
    )
    with torch.inference_mode():
        return model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=256,
        )

def format_response(response: GreedySearchDecoderOnlyOutput) -> str:
    decoded_output = tokenizer.decode(response.sequences[0])
    response = decoded_output.split("### Response:")[1].strip()
    return "\n".join(textwrap.wrap(response))

def ask_alpaca(prompt: str, model: PeftModel = model) -> str:
    prompt = create_prompt(prompt)
    response = generate_response(prompt, model)
    print(format_response(response))

Loading checkpoint shards: 100%|██████████| 3/3 [00:22<00:00,  7.54s/it]


In [47]:
ask_alpaca('유튜브 내달 2일까지 크리에이터 지원 공간 운영 ')

- "우리는 유튜브 내달 2일까지 크리에이터를 지원할 예정입니다."</s>


In [40]:
torch.cuda.empty_cache()